In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aerial-cactus-identification/sample_submission.csv
/kaggle/input/aerial-cactus-identification/train.zip
/kaggle/input/aerial-cactus-identification/test.zip
/kaggle/input/aerial-cactus-identification/train.csv


In [61]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [63]:
device

device(type='cuda')

In [64]:
import pandas as pd

data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [65]:
from zipfile import ZipFile

with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()

with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [66]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(labels,
                               test_size=0.1,
                               stratify=labels['has_cactus'],
                               random_state=50)

In [67]:
print(len(train))
print(len(valid))

15750
1750


## define Dataset & DataLoader

In [68]:
import cv2
from torch.utils.data import Dataset

In [69]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]

        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [70]:
from torchvision import transforms

transform = transforms.ToTensor()

In [71]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

In [72]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

In [73]:
import torch.nn as nn
import torch.nn.functional as F

## CNN Model Version 1
### loss : Cross Entropy
### optimizer : SGD lr : 0.01
### epoch : 10

In [74]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,
                              kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,
                              kernel_size=3, padding=2)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        self.fc = nn.Linear(in_features=64*4*4, out_features=2)

    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x

In [75]:
model = Model().to(device)

In [76]:
criterion = nn.CrossEntropyLoss()

In [77]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [78]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0

    for images, labels in loader_train:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)

        epoch_loss += loss.item()
        loss.backward()

        optimizer.step()

    print(f'epoch [{epoch+1}/{epochs}] loss : {epoch_loss/len(loader_train):.4f}')

epoch [1/10] loss : 0.5233
epoch [2/10] loss : 0.3453
epoch [3/10] loss : 0.2370
epoch [4/10] loss : 0.1984
epoch [5/10] loss : 0.1754
epoch [6/10] loss : 0.1666
epoch [7/10] loss : 0.1545
epoch [8/10] loss : 0.1439
epoch [9/10] loss : 0.1333
epoch [10/10] loss : 0.1299


In [79]:
from sklearn.metrics import roc_auc_score

true_list = []
preds_list = []

In [81]:
model.eval()

with torch.no_grad():
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu()

        preds_list.extend(preds)
        true_list.extend(true)

print(f'ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

ROC AUC : 0.9902


In [82]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

In [88]:
model.eval()

preds = []

with torch.no_grad():
    for images, _ in loader_test:
        images = images.to(device)

        outputs = model(images)

        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()

        preds.extend(preds_part)

In [89]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [90]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')